In [2]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [3]:
path = r'C:\Users\emman\OneDrive\Documents\Instacart Basket Analysis (Oct-22)'

In [4]:
ord_prods_cust_merged = pd.read_pickle(os.path.join(path, '02 Data', 'prepared Data', 'ord_prods_cust_merged.pkl'))

In [5]:
df = ord_prods_cust_merged

In [6]:
# To create an activity flag based on max_order
df.loc[df['max_order']>=5,'activity']='normal'
df.loc[df['max_order']<5,'activity']='low'

In [7]:
# creating a dataframe with only normal activity customers
df2=df[df['activity']=='normal']

In [8]:
df2.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,last_name,gender,state,age,date_joined,no_of_dependents,marital_status,income,activity
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,normal
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,normal
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,normal
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,normal
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,normal


In [9]:
df2.shape

(30964564, 35)

### Q5
 
The marketing and business strategy units at Instacart want to create more-relevant marketing strategies for different products and are, thus, curious about customer profiling in their database. Create a profiling variable based on age, income, certain goods in the “department_id” column, and number of dependents. You might also use the “orders_day_of_the_week” and “order_hour_of_day” columns if you can think of a way they would impact customer profiles. (Hint: As an example, try thinking of what characteristics would lead you to the profile “Single adult” or “Young parent.”)

#### 1. Age Profiling 

 - Customers age less than 35 years old = young adults
 
 - Customers age greater than 35 years old and less than 65 years old = middle aged adult
 
 - Customers age greater than 65 = old adults 

In [10]:
df2.loc[df['age'] <=35, 'age_group'] = 'Young adult'

C:\Users\emman\AppData\Local\Temp\ipykernel_1756\703196721.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[df['age'] <=35, 'age_group'] = 'Young adult'


In [11]:
df2.loc[(df2['age'] <65) & (df2['age'] >35), 'age_group'] = 'Middle-aged adult'

In [12]:
df2.loc[df2['age'] >=60, 'age_group'] = 'Old adult'

In [13]:
# Aggregate for age group useage frequency

df2.groupby('age_group').agg({'median_days_since_prior_order':['mean', 'min','max']})

median_days_since_prior_order           
                                           mean  min   max
age_group                                                 
Middle-aged adult                      9.955354  0.0  30.0
Old adult                              9.950940  0.0  30.0
Young adult                            9.967023  0.0  30.0

In [14]:
# Aggregate for age group expenditure 
df2.groupby('age_group').agg({'prices':['mean', 'min','max']})

prices           
                       mean  min   max
age_group                             
Middle-aged adult  7.790941  1.0  25.0
Old adult          7.795446  1.0  25.0
Young adult        7.784246  1.0  25.0

#### 2. Income Profiling 

 - Customers salary less than 100,000 = low income
 
 - Customers salary between 100,000 - 200,000 = middle income
 
 - Customers salary between 200,000 - 400,000 = upper middle income
 
 - Customers salary between >400,000 = high income

In [15]:
df2.loc[df2['income']<=100000, 'income_group'] = 'low income'

C:\Users\emman\AppData\Local\Temp\ipykernel_1756\1574396364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[df2['income']<=100000, 'income_group'] = 'low income'


In [16]:
df2.loc[(df2['income']>100000) & (df2['income'] <= 200000), 'income_group'] = 'middle income'

In [17]:
df2.loc[(df2['income']>200000) & (df2['income']<=400000), 'income_group'] = 'upper middle income'

In [18]:
df2.loc[df2['income'] >400000, 'income_group'] = 'high income'

In [19]:
df2['income_group'].value_counts (dropna = False)

low income             16757536
middle income          13952886
upper middle income      207146
high income               46996
Name: income_group, dtype: int64

In [20]:
# Aggregate for income group useage frequency

df2.groupby('income_group').agg({'median_days_since_prior_order':['mean', 'min','max']})

median_days_since_prior_order           
                                             mean  min   max
income_group                                                
high income                              9.251894  1.0  30.0
low income                              10.020698  0.0  30.0
middle income                            9.897683  0.0  30.0
upper middle income                      8.980347  1.0  30.0

In [21]:
# Aggregate for age group expenditure 
df2.groupby('income_group').agg({'prices':['mean', 'min','max']})

prices           
                         mean  min   max
income_group                            
high income          7.738014  1.0  25.0
low income           7.652571  1.0  25.0
middle income        7.956613  1.0  25.0
upper middle income  7.786333  1.0  25.0

#### 3. Profiling department_id

In [22]:
# Importing departments dataframe
df_dept = pd.read_csv(os.path.join(path, '02 Data', 'prepared Data', 'department_wrangled.csv'))

In [23]:
df_dept.head()

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [24]:
# Renaming Unnamed:0 column to department_id
df_dept.rename(columns ={'Unnamed: 0': 'department_id'}, inplace = True)

In [25]:
df_dept.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [26]:
# Turning department dataframe into data dictionary
zipped=zip(df_dept['department_id'], df_dept['department'])
data_dict=dict(zipped)
data_dict

{1: 'frozen',
 2: 'other',
 3: 'bakery',
 4: 'produce',
 5: 'alcohol',
 6: 'international',
 7: 'beverages',
 8: 'pets',
 9: 'dry goods pasta',
 10: 'bulk',
 11: 'personal care',
 12: 'meat seafood',
 13: 'pantry',
 14: 'breakfast',
 15: 'canned goods',
 16: 'dairy eggs',
 17: 'household',
 18: 'babies',
 19: 'snacks',
 20: 'deli',
 21: 'missing'}

In [27]:
# Connect department names to department_id dataframe
df2['department'] = df2['department_id'].map(data_dict)

C:\Users\emman\AppData\Local\Temp\ipykernel_1756\1460190409.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['department'] = df2['department_id'].map(data_dict)


In [28]:
df2.head(60)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,state,age,date_joined,no_of_dependents,marital_status,income,activity,age_group,income_group,department
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Alabama,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages


In [29]:
# Creating babay department flag column

df2['baby_status'] = [1 if x == 'babies' else 0 for x in df2['department']]

C:\Users\emman\AppData\Local\Temp\ipykernel_1756\2493960072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['baby_status'] = [1 if x == 'babies' else 0 for x in df2['department']]


In [30]:
df2.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,age,date_joined,no_of_dependents,marital_status,income,activity,age_group,income_group,department,baby_status
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages,0
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages,0
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages,0
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages,0
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,31,2/17/2019,3,married,40423,normal,Young adult,low income,beverages,0


In [31]:
# Grouping and aggregatng userid by baby_status
df2['baby_status'] = df2.groupby(['user_id']) ['baby_status'].transform(np.max)

C:\Users\emman\AppData\Local\Temp\ipykernel_1756\2053788550.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['baby_status'] = df2.groupby(['user_id']) ['baby_status'].transform(np.max)


In [32]:
# Checking column baby_status
df2['baby_status'].value_counts(dropna= False)

0    21154311
1     9810253
Name: baby_status, dtype: int64

In [33]:
# Aggregate for baby_status useage frequency

df2.groupby('baby_status').agg({'median_days_since_prior_order':['mean', 'min','max']})

median_days_since_prior_order           
                                     mean  min   max
baby_status                                         
0                               10.731215  0.0  30.0
1                                8.287965  0.0  30.0

In [34]:
# Aggregate for age group expenditure 
df2.groupby('baby_status').agg({'prices':['mean', 'min', 'max']})

prices           
                 mean  min   max
baby_status                     
0            7.770758  1.0  25.0
1            7.833353  1.0  25.0

#### 3. Profiling frozen

In [35]:
# creating personal care flag

df2['frozen_status'] = [1 if x == 'frozen' else 0 for x in df2['department']]

C:\Users\emman\AppData\Local\Temp\ipykernel_1756\3975242624.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['frozen_status'] = [1 if x == 'frozen' else 0 for x in df2['department']]


In [36]:
# Printing frequency of frozen_status column
df2['frozen_status'].value_counts(dropna = False)

0    28842833
1     2121731
Name: frozen_status, dtype: int64

In [37]:
# Aggregate for frozen_status useage frequency

df2.groupby('frozen_status').agg({'median_days_since_prior_order':['mean', 'min','max']})

median_days_since_prior_order           
                                       mean  min   max
frozen_status                                         
0                                  9.912158  0.0  30.0
1                                 10.568624  0.0  30.0

In [38]:
# Aggregate frozen_status expenditure 
df2.groupby('frozen_status').agg({'prices':['mean', 'min','max']})

prices           
                   mean  min   max
frozen_status                     
0              7.794774  1.0  25.0
1              7.733725  1.0  15.0

#### Profiling number of dependants

 - n_dependants = 0, single household
 - n_dependants between 1-2, small household
 - n_dependants between 2-3, large household

In [39]:
df2.loc[df2['no_of_dependents'] <=0, 'household_size'] = 'single household'

C:\Users\emman\AppData\Local\Temp\ipykernel_1756\3124181809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.loc[df2['no_of_dependents'] <=0, 'household_size'] = 'single household'


In [40]:
df2.loc[(df2['no_of_dependents']>1) & (df2['no_of_dependents'] <= 2), 'household_size'] = 'small family'

In [41]:
df2.loc[(df2['no_of_dependents'] >2) & (df2['no_of_dependents'] <= 3), 'household_size'] = 'medium family'

In [42]:
# Frequency of household_size column to make sure that the household grouping is updated 
df2['household_size'].value_counts(dropna = False)

medium family       7772516
single household    7739681
small family        7733261
NaN                 7719106
Name: household_size, dtype: int64

In [43]:
# Aggregate for household_size useage frequency

df2.groupby('household_size').agg({'median_days_since_prior_order':['mean', 'min','max']})

median_days_since_prior_order           
                                          mean  min   max
household_size                                           
medium family                         9.949826  0.0  30.0
single household                      9.936569  0.0  30.0
small family                          9.972207  0.0  30.0

In [44]:
# Aggregate for household_size expenditure 
df2.groupby('household_size').agg({'prices':['mean', 'min','max']})

prices           
                      mean  min   max
household_size                       
medium family     7.792886  1.0  25.0
single household  7.789550  1.0  25.0
small family      7.784014  1.0  25.0

In [48]:
df2.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'normal_data.pkl'))